In [1]:
pip install bigquery

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.1/215.1 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/

  DEPRECATION: dacktool is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for dacktool ... done
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.3
    Uninstalling pandas-1.3.3:
      Successfully uninstalled pandas-1.3.3
  DEPRECATION: google-api-python-client is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for google-api-python-client ... done
  DEPRECATION: dbstrea

In [24]:
pip install pandas_gbq


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install db_dtypes


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install 'google-cloud-bigquery[pandas]'


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import pandas_gbq as pd_gbq
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

## Create GBQ Client from credentials

In [3]:
### Create API Key in GCP Console (Remove)
api_key = '<insert api key>'

### Create GCP Service Account (Remove)
service_account_email = 'maxschlafli-smg-service-accoun@smgmaxschlafli.iam.gserviceaccount.com'

### Create client using json file & project_id
client = bigquery.Client.from_service_account_json(json_credentials_path="credentials.json")

## Create test dataset

In [4]:
project = 'smgmaxschlafli'
dataset_id = f"{project}.test_dataset"

In [20]:
# Create dataset object
dataset = bigquery.Dataset(dataset_id)

# Set geo location
dataset.location = "EU"

dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print(f"Created dataset {dataset_id}")

Created dataset smgmaxschlafli.test_dataset


## Create test table

In [5]:
table_id = f"{dataset_id}.test_table"

In [22]:
schema = [
    bigquery.SchemaField("full_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("age", "INTEGER", mode="REQUIRED"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  
print(f"Created table {table_id}")

Created table smgmaxschlafli.test_dataset.smgmaxschlafli.test_dataset.test_table


## Load test DF into BGQ table

In [29]:
## Create DF
df = pd.DataFrame(
    {
        "full_name": ["max", "markus", "michael", "melanie"],
        "age": [29, 32, 34, 36],
    }
)

### Load DF to test table
try:
    pd_gbq.to_gbq(df, table_id, project_id=project)
    print(f"Populated table {dataset_id}.{table_id}")

except Exception as e:
    print(e)

Reason: Already Exists: Table smgmaxschlafli:test_dataset.test_table


In [6]:
sql = """
SELECT * FROM `smgmaxschlafli.test_dataset.test_table` LIMIT 5
"""

df_gbq = client.query(sql).to_dataframe()

In [7]:
df_gbq

,full_name,age
0,max,29
1,markus,32
2,michael,34
3,melanie,36
